Do a quick check on the proportion of 90% events both annually and seasonally

In [1]:
# Import packages
import numpy as np
import pandas as pd
import xarray as xr
import datetime


In [2]:
#directories
iDir = "~/PROGS/Belmont FWE/LOCA-data/loca5/"
oDir = "~/PROGS/Belmont FWE/Indices/"

Define the precipitation functions

In [3]:

def r_quant(ds, threshold=0.95, time0='2015-01-01', time1='2034-12-31', varname='PRECT'):
    """
    Compute quantiles in rainfall data. Calculates for days with >=1mm.
    Args:
        ds: Dataset.
        varname: PRECT (assumes CESM output)
        threshold (float): Lower quantile percent (as decimal). Defaults to 0.95
        time0 (str): First time for slice. Defaults to 2015-01-01.
        time1 (str): Second time for slice. Defaults to 2034-12-31.
    """
    dp = ds.sel(time=slice(time0,time1)).where(ds[varname] > 1).quantile(q=[threshold],dim=['time']).squeeze().drop('quantile')
    dp = dp.to_array(name=(f"Q{str(int(float(threshold)*100))}"))
    return dp

def annualtot_above_q(ds, thresh_data, threshold=0.95, varname='PRECT'):
    """
    Compute total precipitation from days exceeding threshold
    Args:
        ds: Dataset.
        thresh_data: Threshold dataset (computed using r_quant over all projs).
        threshold (float): Upper quantile percent (as decimal) defaults to 0.95
        varname: PRECT. assumes CESM output
    """
    ds = xr.where(ds[varname] > thresh_data,ds[varname],0).groupby('time.year').sum('time')
    ds = ds.to_array(name = (f"PR{str(int(float(threshold)*100))}Tot")).squeeze().drop('variable')
    return ds

def annualnum_above_q(ds, thresh_data, threshold=0.95, varname='PRECT'):
    """
    Compute number of days exceeding threshold per year
    Args:
        ds: Dataset
        thresh_data: Threshold dataset (computed using r_quant over all ensemble projs)
        threshold: upper quantile percent as decimal, defaults to 0.95
        varname: PRECT assumes CESM output
    """
    ds = xr.where(ds[varname] > thresh_data,1,0).groupby('time.year').sum('time')
    ds = ds.to_array(name = (f"N{str(int(float(threshold)*100))}")).squeeze().drop('variable')
    return ds


In [4]:
with xr.open_dataset(iDir + 'Extraction_pr.nc') as dp:
    dp
    
dp_win = dp.isel(time=(dp.time.dt.season=='DJF'))
dp_win['win_year'] = (dp_win.time.dt.month >= 12) + dp_win.time.dt.year
dp_win.coords['win_year'] = dp_win['win_year']
dp_spr = dp.isel(time=(dp.time.dt.season=='MAM'))
dp_sum = dp.isel(time=(dp.time.dt.season=='JJA'))
dp_aut = dp.isel(time=(dp.time.dt.season=='SON'))

In [43]:
dclim = dp.sel(time=slice('1981','2010'))
years = np.arange(1981,2011)

lat = np.linspace(39.53125,42.34375,num=46)
lon = np.linspace(283.78125,285.59375, num=30)
dims = ('year', 'lat', 'lon')

In [65]:
# Keeps warning about the blank grid cells.
# Calculates quantile over climatology using leave-one-out bootstrap
quans = xr.DataArray(None, coords=dict(year=years, lat = lat, lon = lon), dims=dims)
for year in years:
    tdrop = dclim.where(dclim['time.year']!=year)
#    print(year)
    quans[:,:,:]=tdrop.where(tdrop['pr']>1).quantile(q=0.95, dim=('time','projection'), skipna=True).squeeze().to_array()

/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/glade/u/home/maritye/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/glade/u/h

In [66]:
dq = quans.mean(dim='year').squeeze()

/glade/u/home/maritye/.local/lib/python3.7/site-packages/xarray/core/nanops.py:133: RuntimeWarning: invalid value encountered in true_divide
  data = data / (valid_count - ddof)


Now calculate the total from these events and number of days in each year for annual total and then seasonal values.

In [76]:
### Create some standard information to include in each of the datasets for ease of writing
years = np.arange(1980,2100)
winyears = np.arange(1981,2099)
seasons = ('DJF', 'MAM', 'JJA', 'SON')
proj= ['cesm1-cam5.1.rcp45', 'cesm1-cam5.1.rcp85']
lat = np.linspace(39.53125,42.34375,num=46)
#lon = np.linspace(-76.21875,-74.40625, num=30)
lon = np.linspace(283.78125,285.59375, num=30) # calculate with original longs and then convert later
dims = ("proj", 'year', 'lat', 'lon')
attribs = dict(description='High Threshold Precipitation Indices for LOCA data over Delaware River Basin. Using CESM1/RCP4.5 and RCP8.5.', 
                history='Created by Mari Tye December 2022.' )

In [113]:
P95T = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='P95T')
PTOT = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='PRCPTOT')
N95 = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='N95')

for r in range(2):
    print(proj[r])
    
    PTOT[r,:,:,:] = xr.DataArray(dp['pr'].isel(projection=r).groupby('time.year').sum('time'))
    dum = dp.where(dp['pr']>dq).isel(projection=r).groupby('time.year').sum()
    P95T[r,:,:,:] = dum['pr']
    N95[r,:,:,:] = xr.where(dp['pr']>dq,1,0).isel(projection=r).groupby('time.year').sum()

cesm1-cam5.1.rcp45
cesm1-cam5.1.rcp85


In [119]:
ptotnm = oDir + 'LOCA_CESM1.PRCPTOT.1981-2099.nc'
p95nm = oDir + 'LOCA_CESM1.P95T.1981-2099.nc'
n95nm = oDir + 'LOCA_CESM1.N95.1981-2099.nc'
   
        # and write out to netcdf
P95T.to_netcdf(p95nm)
N95.to_netcdf(n95nm)
PTOT.to_netcdf(ptotnm)


In [124]:
P95TW = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='P95T')
N95W = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='N95')

for r in range(2):
    print(proj[r])
    
    dum = dp_win.where(dp_win['pr']>dq).isel(projection=r).groupby('win_year').sum()
    P95TW[r,:,:,:] = dum['pr'][:120,:,:]
    N95W[r,:,:,:] = xr.where(dp_win['pr']>dq,1,0).isel(projection=r).groupby('win_year').sum()[:120,:,:]

cesm1-cam5.1.rcp45
cesm1-cam5.1.rcp85


In [125]:
P95TS = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='P95T')
N95S = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='N95')

for r in range(2):
    print(proj[r])
    
    dum = dp_sum.where(dp_sum['pr']>dq).isel(projection=r).groupby('time.year').sum()
    P95TS[r,:,:,:] = dum['pr']
    N95S[r,:,:,:] = xr.where(dp_sum['pr']>dq,1,0).isel(projection=r).groupby('time.year').sum()

cesm1-cam5.1.rcp45
cesm1-cam5.1.rcp85


In [127]:
P95TG = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='P95T')
N95G = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='N95')

for r in range(2):
    print(proj[r])
    
    dum = dp_spr.where(dp_spr['pr']>dq).isel(projection=r).groupby('time.year').sum()
    P95TG[r,:,:,:] = dum['pr']
    N95G[r,:,:,:] = xr.where(dp_spr['pr']>dq,1,0).isel(projection=r).groupby('time.year').sum()

cesm1-cam5.1.rcp45
cesm1-cam5.1.rcp85


In [128]:
P95TA = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='P95T')
N95A = xr.DataArray(None, coords=dict(proj=proj, year=years, lat = lat, lon = lon), dims=dims, attrs=attribs, name='N95')

for r in range(2):
    print(proj[r])
    
    dum = dp_aut.where(dp_aut['pr']>dq).isel(projection=r).groupby('time.year').sum()
    P95TA[r,:,:,:] = dum['pr']
    N95A[r,:,:,:] = xr.where(dp_aut['pr']>dq,1,0).isel(projection=r).groupby('time.year').sum()

cesm1-cam5.1.rcp45
cesm1-cam5.1.rcp85


In [129]:
p95wnm = oDir + 'LOCA_CESM1.P95T.1981-2099.DJF.nc'
n95wnm = oDir + 'LOCA_CESM1.N95.1981-2099.DJF.nc'
p95gnm = oDir + 'LOCA_CESM1.P95T.1981-2099.MAM.nc'
n95gnm = oDir + 'LOCA_CESM1.N95.1981-2099.MAM.nc'
p95snm = oDir + 'LOCA_CESM1.P95T.1981-2099.JJA.nc'
n95snm = oDir + 'LOCA_CESM1.N95.1981-2099.JJA.nc'
p95anm = oDir + 'LOCA_CESM1.P95T.1981-2099.SON.nc'
n95anm = oDir + 'LOCA_CESM1.N95.1981-2099.SON.nc'
  
        # and write out to netcdf
P95TW.to_netcdf(p95wnm)
N95W.to_netcdf(n95wnm)
P95TG.to_netcdf(p95gnm)
N95G.to_netcdf(n95gnm)
P95TS.to_netcdf(p95snm)
N95S.to_netcdf(n95snm)
P95TA.to_netcdf(p95anm)
N95A.to_netcdf(n95anm)
